In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict

dataset_path = '/Users/rjaditya/Documents/NEU-SEM/Spring-2024/ML/LLM-vs-Human-Text-Detection/train_v2_drcat_02.csv'
df = pd.read_csv(dataset_path, header = 0)

### get the labels
label = df['label']
label = label.astype("int")
# df.drop(columns='label', axis = 1, inplace = True)
df.drop(columns='RDizzl3_seven', axis = 1, inplace=True)
df.drop(columns='prompt_name', axis = 1, inplace=True)
df.drop(columns='source', axis = 1, inplace=True)
df.dropna(subset=['text'], inplace=True)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)

/opt/homebrew/anaconda3/envs/ML/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
test_df.head()

,text,label
39024,"Ummm... hey there! So, like, I know it's kind...",1
16614,"Dear Principal,\n\nAs you may know, many stude...",0
37944,The cafeteria at our school is an important re...,1
3563,Some schools require students to complete summ...,0
31644,When considering the advantages and disadvanta...,1


In [6]:
# create a dataset object with train and test data.
dataset = DatasetDict()
dataset['train'] = Dataset.from_pandas(train_df)
dataset['test'] = Dataset.from_pandas(test_df)
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 35894
    })
    test: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 8974
    })
})


In [ ]:
from transformers import AlbertTokenizer, AlbertModel
import torch

### testing
model = AlbertModel.from_pretrained("albert-base-v2", num_labels = 2)
tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")

In [8]:
def tokenize_func(dataset):
    return tokenizer(dataset['text'], padding = "max_ength", truncation = True)

tokenized_datasets = dataset.map(tokenize_func, batched = True)
train_dataset = tokenized_datasets["train"].shuffle(seed=42)
eval_dataset = tokenized_datasets["test"].shuffle(seed=42)


In [ ]:
from transformers import Trainer, TrainingArguments
import evaluate

args = TrainingArguments(output_dir="LLMvHuman", evaluation_strategy="epoch")
metric = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

for epoch in range(2):
    trainer.train()

while torch.no_grad():
    essay = """
            this is bla bla generated by bla bla. will my model bla bla should i find bla bla. Rombo bla bla adikuromo. Adippom.
        """
    op = model()
    prediction = F.softmax(op.logits, dim = 1)
    labels = torch.argmax(prediction)
    print(label)